In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

# Check if CUDA is available and set device to GPU if it is
dgpu = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [2]:
# Read the MPG dataset.
csvPath = "trainset.csv"
df = pd.read_csv(csvPath,header=None)

In [3]:
# Pandas to Numpy
x = df.iloc[:,range(24)].values
y = df.iloc[:,[24,25]].values  # regression

# Numpy to PyTorch
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

In [4]:
print(f"Data: x size ={x.shape}")
print(f"Label: y size ={y.shape}")

Data: x size =torch.Size([9000, 24])
Label: y size =torch.Size([9000, 2])


In [5]:
# Create the MLP model
model = nn.Sequential(
    nn.Linear(x.shape[1], 18),
    nn.ReLU(),
    nn.Linear(18, 8),
    nn.ReLU(),
    nn.Linear(8, 2)
)
model = model.to(dgpu)

In [6]:
# PyTorch 2.0 Model ahead-of-time (AOT) compilation using the eager mode backend.
model = torch.compile(model,backend="aot_eager")

# Define the loss function for regression
loss_mse = nn.MSELoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)   # Use Adam optimizer

In [7]:
x = x.to(dgpu)
y = y.to(dgpu)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)   # Use Adam optimizer
# Train for 100,000 epochs.
for epoch in range(100000):
    optimizer.zero_grad()
    out = model(x)
    loss = loss_mse(out, y)
    loss.backward()
    optimizer.step()

    # Display status every 100 epochs.
    if epoch % 10000 == 0:
        print(f"Epoch {epoch}, loss: {loss.item()}")

Epoch 0, loss: 4089.818359375
Epoch 10000, loss: 10.351186752319336
Epoch 20000, loss: 8.503694534301758
Epoch 30000, loss: 7.91792106628418
Epoch 40000, loss: 7.414238929748535
Epoch 50000, loss: 7.2318549156188965
Epoch 60000, loss: 7.095175266265869
Epoch 70000, loss: 6.889412879943848
Epoch 80000, loss: 6.836796283721924
Epoch 90000, loss: 6.57572078704834


In [8]:
pred = model(x)
print(f"Shape: {pred.shape}")
print(pred[0:10])

Shape: torch.Size([9000, 2])
tensor([[35.0943, 45.9024],
        [36.3593, 47.9790],
        [77.7868, 75.0867],
        [46.2200, 60.1785],
        [77.5696, 76.8736],
        [39.2631, 52.7232],
        [74.7539, 83.4317],
        [64.0380, 71.5518],
        [55.8704, 61.9317],
        [77.0808, 74.0154]], device='cuda:0', grad_fn=<SliceBackward0>)


In [9]:
from sklearn import metrics

# Measure RMSE error.  RMSE is common for regression.

score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print(f"Final score (RMSE): {score}")

Final score (RMSE): 2.6175897121429443


In [10]:
score = torch.sqrt(torch.nn.functional.mse_loss(pred, y))
print(f"Final score (RMSE) with training dataset: {score}")

Final score (RMSE) with training dataset: 2.617591619491577


In [46]:
# Sample predictions
for i in range(50):
    print(f"{i+1}. Actual: {y[i]}, " + f"Predicted: {pred[i]}")

1. Actual: tensor([32.9114, 43.6894], device='cuda:0'), Predicted: tensor([33.3982, 44.1787], device='cuda:0', grad_fn=<SelectBackward0>)
2. Actual: tensor([33.5422, 43.5609], device='cuda:0'), Predicted: tensor([35.0445, 44.8536], device='cuda:0', grad_fn=<SelectBackward0>)
3. Actual: tensor([77.3071, 75.7072], device='cuda:0'), Predicted: tensor([76.1236, 75.9842], device='cuda:0', grad_fn=<SelectBackward0>)
4. Actual: tensor([46.6223, 60.6976], device='cuda:0'), Predicted: tensor([46.7042, 61.7231], device='cuda:0', grad_fn=<SelectBackward0>)
5. Actual: tensor([77.0109, 77.1023], device='cuda:0'), Predicted: tensor([76.7840, 76.9242], device='cuda:0', grad_fn=<SelectBackward0>)
6. Actual: tensor([39.9643, 50.4153], device='cuda:0'), Predicted: tensor([37.8064, 49.8892], device='cuda:0', grad_fn=<SelectBackward0>)
7. Actual: tensor([78.2679, 73.2748], device='cuda:0'), Predicted: tensor([74.7063, 77.4578], device='cuda:0', grad_fn=<SelectBackward0>)
8. Actual: tensor([62.4639, 71.156

In [55]:
csvPath = "testset.csv"
df_test = pd.read_csv(csvPath)
# Pandas to Numpy
xt = df_test.iloc[:,range(24)].values
yt = df_test.iloc[:,[24,25]].values  # regression

# Numpy to PyTorch
xt = torch.tensor(xt, dtype=torch.float32).to(dgpu)
yt = torch.tensor(yt, dtype=torch.float32)

# Pass into model
yt_pred = model(xt).cpu()

In [58]:
score = torch.sqrt(torch.nn.functional.mse_loss(yt_pred, yt))
print(f"Final score (RMSE) with test set: {score}")

Final score (RMSE) with test set: 1.9144036769866943


In [59]:
import csv
with open('pred.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(yt_pred)

In [60]:
for i in range(10):
    print(yt[i],yt_pred[i])

tensor([53.4241, 63.2120]) tensor([52.5213, 63.5209], grad_fn=<SelectBackward0>)
tensor([36.6702, 45.9392]) tensor([35.8287, 45.3862], grad_fn=<SelectBackward0>)
tensor([57.8885, 61.5989]) tensor([59.2745, 61.6897], grad_fn=<SelectBackward0>)
tensor([80.0552, 75.1403]) tensor([75.7263, 77.0635], grad_fn=<SelectBackward0>)
tensor([63.3119, 76.4552]) tensor([64.2205, 75.9043], grad_fn=<SelectBackward0>)
tensor([75.9189, 72.5088]) tensor([72.8755, 71.9478], grad_fn=<SelectBackward0>)
tensor([55.9193, 63.1268]) tensor([55.3061, 63.6761], grad_fn=<SelectBackward0>)
tensor([75.9794, 76.5926]) tensor([76.1591, 78.1346], grad_fn=<SelectBackward0>)
tensor([37.5860, 47.9917]) tensor([38.3881, 49.5250], grad_fn=<SelectBackward0>)
tensor([64.5485, 60.7042]) tensor([65.1673, 60.7156], grad_fn=<SelectBackward0>)
